In [46]:
require('nn')
require('hdf5')
require('optim')

In [47]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [50]:
function bigramCount(X, K)
    local countMatrix = torch.zeros(K, K)
    for i=1,X:size(1) do
        for j=2,X:size(2) do
            local prev_word = X[i][j-1]
            local curr_word = X[i][j]
            countMatrix[prev_word][curr_word] = countMatrix[prev_word][curr_word] + 1
        end
    end
    return CM
end

function unigramCount(X, Y, K)
    local countVector = torch.zeros(1, K)
    for i=1,X:size(1) do
        for j=2,X:size(2) do
            local idx = X[i][j]
            countVector[1][idx] = countVector[1][idx] + 1
        end
    end
    return countVector
end

In [40]:
function additive(matrix, alpha)
    return matrix:add(alpha)
end

function normalize(CV, CM)
    return CM:cdiv(torch.expand(CV:t(), CM:size(1), CM:size(2)))
end

function test(bcm, valid, kaggle, q)
    p = 0
    for i=1,valid:size(1) do
        idx = q[i][kaggle[i]]
        prev = valid[i][5]
        p = torch.log(bcm[prev][idx]) + p
    end
    return torch.exp(-p/X_valid:size(1))
end

function kaggle(bcm, test, q)
    scores = torch.zeros(q:size(1), 50)
    for i=1,test:size(1) do
        prev = test[i][5]
        for j=1,50 do
            w = q[i][j]
            scores[i][j] = bcm[prev][w]
        end
    end
    return scores:cdiv(torch.expand((torch.sum(scores, 2)), scores:size(1), scores:size(2)))
end

function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [51]:
CM = bigramCount(X_train, nfeatures)
CM = additive(CM, 1)

In [ ]:
CV = unigramCount(X_train, Y_train, nfeatures)
CV = additive(CV, 1)

In [ ]:
BCM = normalize(CV, CM)

In [ ]:
JLL = distribution(BCM, X_valid)

In [55]:
valid_perplexity(JLL, kaggle_valid, q_valid)

-33413.805373877	
20233.112593302	



In [41]:
scores = kaggle(BCM, X_test, q_test)

In [42]:
write2file(scores, "asdf.csv")

In [37]:
test(BCM, X_valid, kaggle_valid, q_valid)

133.14713426227	
